In [1]:
!pip install numpy opencv-python matplotlib pillow


In [2]:
!pip install tensorflow keras


In [3]:
!pip install -U efficientnet
import efficientnet.tfkeras as efn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00


In [4]:
!pip install scikit-image


In [5]:
!pip install pandas


In [6]:
import cv2
import sys
import shutil
import random
import os
import csv
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
from PIL import Image
import pandas as pd
import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
#!pip install efficientnet
from efficientnet.tfkeras import EfficientNetB0
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop


In [ ]:
#create necessary directory if the directory doesn't exist
def create_necessary_folder():
    #make dir for plottings
    if not os.path.exists('plottings'): os.mkdir(f'{os.getcwd()}\\plottings')
    #subfolders for every architecture
    if not  os.path.exists('plottings\\vgg16'): os.mkdir(f'{os.getcwd()}\\plottings\\vgg16')
    if not  os.path.exists('plottings\\vgg19'): os.mkdir(f'{os.getcwd()}\\plottings\\vgg19')
    if not  os.path.exists('plottings\\effnet'): os.mkdir(f'{os.getcwd()}\\plottings\\effnet')
    #make dir for models
    if not os.path.exists('models'): os.mkdir(f'{os.getcwd()}\\models')
    #subfolders for every architecture
    if not  os.path.exists('models\\vgg16'): os.mkdir(f'{os.getcwd()}\\models\\vgg16')
    if not  os.path.exists('models\\vgg19'): os.mkdir(f'{os.getcwd()}\\models\\vgg19')
    if not  os.path.exists('models\\effnet'): os.mkdir(f'{os.getcwd()}\\models\\effnet')


create_necessary_folder()


In [ ]:
def step_decay_schedule(initial_lr=1e-3, decay_factor=0.75, step_size=10):
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch / step_size))
    return LearningRateScheduler(schedule)


def import_dataset(path):
    im = Image.open(path).convert('RGB')
    return im


### Try EfficientNet B0 (the base model)

In [ ]:
X, Y = [], []

dataset = pd.read_csv('ground-truth.csv')
EPOCHS = 15
LEARNING_RATE = 1e-3

for index, row in dataset.iterrows():
    X.append(array(import_dataset(row[0]).resize((100, 100))).flatten() / 255.0)
    Y.append(row[1])

X = np.array(X)
Y = to_categorical(Y, 2)
X = X.reshape(-1, 100, 100, 3)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.20, random_state=5)

#Load EfficientNet, also set drop_connect_rate for stronger regularization
#input layer
inputs = layers.Input(shape=(100,100,3))
#declare EfficientNetB0
model = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=inputs, input_shape=(100, 100, 3), drop_connect_rate=0.4)
model.trainable = False
#rebuild top layers
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)
top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(2, activation="softmax", name="pred")(x)
model = Model(inputs=inputs, outputs=outputs, name="EfficientNet")
model.summary()

optimizer = optimizers.Adam(lr=LEARNING_RATE, epsilon=None)


#hyperparameter settings
epochs = EPOCHS
batch_size = 16
optimizer = optimizers.Adam(lr=LEARNING_RATE, epsilon=None)
model.compile(optimizer=optimizer, loss="mean_squared_error",metrics=['accuracy'])

lr_sched = step_decay_schedule(initial_lr=1e-4, decay_factor=0.75, step_size=2)


In [ ]:
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, Y_val), verbose=2,callbacks=[lr_sched])


##### Set random ID for current training

In [ ]:
import random
#assign random code to make unique ID
r = random.randint(1, 1000)
print(f'Your unique ID is {r}, refer to this ID to check the latest data!')
ID = f'{LEARNING_RATE}n{EPOCHS}_{r}'

In [ ]:
#plotting
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

#plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.savefig(f'plottings\\effnet\\acc_{ID}.png')
plt.show()

#plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.savefig(f'plottings\\effnet\\loss_{ID}.png')
plt.show()

Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val, axis=1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=range(2))
plt.savefig(f'plottings\\effnet\\cnfmtx_{ID}.png')
plt.show()

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

unfreeze_model(model)

epochs = EPOCHS  # @param {type: "slider", min:8, max:50}
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, Y_val), verbose=2,callbacks=[lr_sched])


In [ ]:
#plotting
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

#plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.savefig(f'plottings\\effnet\\ftacc_{ID}.png')
plt.show()

#plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.savefig(f'plottings\\effnet\\ftloss_{ID}.png')
plt.show()

Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val, axis=1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes=range(2))
plt.savefig(f'plottings\\effnet\\ftcnfmtx_{ID}.png')
plt.show()

In [ ]:
#save model
model_name = f'models\\effnet\\{ID}.h5'
model.save(model_name)


In [ ]:
train_acc, val_acc = np.mean(history.history['accuracy']), np.mean(history.history['val_accuracy'])
print(train_acc, val_acc)

In [ ]:
with open('currently-used-effnet.txt','w+') as f:
    f.write(model_name)

## Testing

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
#test_set = glob.glob('MICC-F220\\*')
test_set = glob.glob('test_set\\*')
classes = []
for img in test_set:
    #data will have the label `tamp` if the data is tampered
    if 'tamp' in img and img[len(img)-4:len(img)] == '.jpg':
        classes.append('forged')
    elif 'tamp' not in img and img[len(img)-4:len(img)] == '.jpg':
        classes.append('unforged')

#for i in range(len(test_set)): print(test_set[i], classes[i])
#def predict():
#load model
model = load_model(model_name)
predictions = []
#test image iteration
for i in tf.range(len(test_set)): #use tf.range for faster execution (operation is done in tf.Tensor)
    image = Image.open(test_set[i]).convert('RGB')
    image = array(image.resize((100,100))).flatten() / 255.0
    image = image.reshape(-1,100,100,3)
    pred = model.predict(image)[0]
    #take the index of the predicted class
    index = np.argmax(pred)
    verdict = "forged" if index == 1 else "unforged"
    predictions.append(verdict)

In [ ]:
correct = 0
for i in range(len(classes)):
    if classes[i] == predictions[i]:
        correct += 1
    else: continue
acc = correct / len(classes)
print(f'Test accuracy: {round(acc*100,3)}%')

In [ ]:
with open(f'plottings\\effnet\\testacc_{ID}.txt','w+') as f:
    f.write(f'train_acc: {train_acc}\nval_acc: {val_acc}\n')
    f.write(f'test_acc: {acc}')

In [ ]:
#for i in range(len(classes)): print(classes[i], predictions[i])
correct = 0
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for i in range(len(classes)):
    im = cv2.imread(test_set[i])
    #im.resize((30,30))
    #print(im.shape)
    plt.subplot(len(classes)/5+1,5,i+1)
    plt.imshow(im)
    color = 'green' if classes[i] == predictions[i] else 'red'
    if color == 'green': correct += 1
    #plt.title(f'{classes[i]}\n{predictions[i]}', color=color)
    plt.text(im.shape[1]/2,0,f'{classes[i]}\n', ha="center", va="bottom", size="medium")
    plt.text(im.shape[1]/2,0,f'\n{predictions[i]}', ha="center", va="bottom", size="medium",color=color)
    plt.axis('off')

plt.savefig(f'plottings\\vgg19\\test_{ID}.png')
plt.show()
